In [1]:
import sys 
print(sys.version)

3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical, pad_sequences
from keras.callbacks import EarlyStopping

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score

import xgboost as xgb
from sklearn.metrics import confusion_matrix

In [3]:
# python version 3.8.6rc1
import pandas as pd
import string
import util
import io
import os
from pprint import pprint
import spacy
import pickle
import re
import numpy as np

import warnings
warnings.filterwarnings(action='once')
#warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
# import pyLDAvis
# import pyLDAvis.gensim_models
# import matplotlib.pyplot as plt
# import matplotlib
# %matplotlib inline
# import pandas as pd
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn import metrics
# from sklearn.decomposition import LatentDirichletAllocation

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wei99\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wei99\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wei99\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\wei99\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
# Count number of rows with each label without considering combination
comments = pd.read_csv("train.csv")
#comments.iloc[:, 2:].sum(axis=0)

df_multilabel = comments

# convert into binary
comments['harmful'] = comments.iloc[:, 2:].sum(axis=1)
comments['harmful'][comments['harmful']>0] = 1

# keep necessary info
df = comments[['comment_text',"harmful"]]#.copy()

print(df['harmful'].value_counts())
df.head()

sns.countplot(df['harmful'])
plt.xlabel('Label')
plt.title('Number of harmful messages')

df_harmful = df[df['harmful'] > 0] 
df_not_harmful = df[df['harmful'] == 0]
df.head()

0    143346
1     16225
Name: harmful, dtype: int64


C:\Users\wei99\AppData\Local\Temp\ipykernel_36132\716027470.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['harmful'][comments['harmful']>0] = 1


,comment_text,harmful
0,Explanation\r\nWhy the edits made under my use...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\r\nMore\r\nI can't make any real suggestions...",0
4,"You, sir, are my hero. Any chance you remember...",0


# Data Cleaning

In [5]:
# Randomly select 15% of not harmful instances + all harmful instances
df_not_harmful_sample = df_not_harmful.sample(frac=0.15)
sample_df = pd.concat([df_not_harmful_sample, df_harmful], axis=0)

In [6]:
def tokenize(text):
    text = text.lower()
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text)
    words = nopunct.split(' ')
    # remove any non ascii
    words = [word.encode('ascii', 'ignore').decode('ascii') for word in words]
    lmtzr = WordNetLemmatizer()
    words = [lmtzr.lemmatize(w) for w in words]
    words = [w for w in words if len(w) > 2]
    return words

In [7]:
train = sample_df.sample(frac=0.80)
test = sample_df.drop(train.index)

## Applying TFIDF

In [8]:
vector = TfidfVectorizer(ngram_range=(1, 1), analyzer='word',
                         tokenizer=tokenize, stop_words='english',
                         strip_accents='unicode', use_idf=1, min_df=10)
X_train = vector.fit_transform(train['comment_text'])
X_test = vector.transform(test['comment_text'])

In [9]:
Y_train = train['harmful']
Y_test = test['harmful']

# Logistic Regression

In [10]:
logistic_regression_classifier = LogisticRegression()

logistic_regression_classifier.fit(X_train, train['harmful'])
predicted = logistic_regression_classifier.predict(X_test)

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay
print("Accuracy of logistic regression:",accuracy_score(test['harmful'], predicted))
print("Recall of logistic regression:",recall_score(test['harmful'], predicted))

Accuracy of logistic regression: 0.8971504307488403
Recall of logistic regression: 0.8295800365185636


## Logistic regression Hyperparameter Tuning

In [12]:
parameter_grid = {'solver': ['newton-cg', 'lbfgs', 'liblinear'],
                  'class_weight': [None, 'balanced']}

cross_validation = StratifiedKFold(n_splits=5)

grid_search = GridSearchCV(logistic_regression_classifier,
                           param_grid=parameter_grid,
                           cv=cross_validation,
                           scoring='f1')

grid_search.fit(X_train, train['harmful'])

print('Best parameters: {}'.format(grid_search.best_params_))

grid_search.best_estimator_

Best parameters: {'class_weight': 'balanced', 'solver': 'liblinear'}


LogisticRegression(class_weight='balanced', solver='liblinear')

In [13]:
lr_gs = LogisticRegression(class_weight='balanced', solver='liblinear')
lr_gs.fit(X_train, train['harmful'])
lrgs_predicted = lr_gs.predict(X_test)
print("Accuracy of logistic regression after Hyperparameter Tuning with Grid Search:",accuracy_score(test['harmful'], lrgs_predicted))
print("Recall of logistic regression after Hyperparameter Tuning with Grid Search:",recall_score(test['harmful'], lrgs_predicted))

Accuracy of logistic regression after Hyperparameter Tuning with Grid Search: 0.9015241882041087
Recall of logistic regression after Hyperparameter Tuning with Grid Search: 0.8627510651247717


In [14]:
conf_matrix = confusion_matrix(y_true=Y_test, y_pred=lrgs_predicted)
#
# Print the confusion matrix using Matplotlib
#
fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()
plt.savefig("logistic.png")

C:\Users\wei99\AppData\Local\Temp\ipykernel_36132\935626199.py:14: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


# Boosting

In [15]:
xgb = xgb.XGBClassifier(objective='binary:logistic',eval_metric='error')

xgb.fit(X_train, train['harmful'])
xgb_prediction = xgb.predict(X_test)
print("Accuracy of XGBoost:",accuracy_score(test['harmful'], xgb_prediction))
print("Recall of XGBoost:",recall_score(test['harmful'], xgb_prediction))

Accuracy of XGBoost: 0.8779324055666003
Recall of XGBoost: 0.759281801582471


In [16]:
conf_matrix_XGB = confusion_matrix(y_true=Y_test, y_pred=xgb_prediction)
#
# Print the confusion matrix using Matplotlib
#
fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix_XGB, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix_XGB.shape[0]):
    for j in range(conf_matrix_XGB.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix_XGB[i, j], va='center', ha='center', size='xx-large')

plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()
plt.savefig("XGBoost_CM.png")

C:\Users\wei99\AppData\Local\Temp\ipykernel_36132\1112707220.py:14: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
